<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="400" alt="cognitiveclass.ai logo"  />
</center>

## Investigation on Saldo  on price BTC, size BTC

## Lab 3. Data Analysis with Python

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Explore features or charecteristics to predict price of cryptocurrency


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li><a href="https://#import_data">Import Data from Module</a></li>
    <li><a href="https://#pattern_visualization">Analyzing Individual Feature Patterns using Visualization</a></li>
    <li><a href="https://#discriptive_statistics">Descriptive Statistical Analysis</a></li>
    <li><a href="https://#basic_grouping">Basics of Grouping</a></li>
    <li><a href="https://#correlation_causation">Correlation and Causation</a></li>
    <li><a href="https://#durbin-watson">Durbin Watson Test</a></li>
    <li><a href="https://#granger">Granger Causality Test</a></li>
</ol>

</div>

<hr>


## What are the main characteristics that have the most impact on the cryptocurrency price?


# 1. Import Data


### Setup


If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:


In [ ]:
#If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:
#install specific version of libraries used in lab
#! mamba install pandas==1.3.3
#! mamba install numpy=1.21.2
#! mamba install scipy=1.7.1-y
#!  mamba install seaborn=0.9.0-y
! mamba install mplfinance -y
! mamba install astropy -y

Import libraries:

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from typing import List
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import grangercausalitytests

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from astropy.visualization import astropy_mpl_style
import itertools
from itertools import combinations
from IPython.display import display

First, we assign the URL of the dataset to <code>"path"</code<>.


In [ ]:
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0MADEN/clean_df%20(4).csv"

Use the Pandas method <code>read_csv()</code> to load the data from the web address. Set the parameter  <code>index_col=0</code> in order to use the first column of cvs file as the index of the dataframe.

In [ ]:
df = pd.read_csv(path, index_col=0)

Set dataframe index column type to <strong>datetime</strong> using <code>pd.to_datetime()</code> method for correct time series analysis. 

In [ ]:
df.index = pd.to_datetime(df.index)

In the previous lab we calculated technical financial indicators. Since the values of previous periods had to be taken into account for their calculation, the first few lines of the dataframe contain `NaN` values.

We will use different methods for recovering missing data in this module that do not work correctly with recovering data in the first rows of time series. Therefore, we need to remove `NaN` values with `df.dropna(inplace=True)` method.

In [ ]:
df.dropna(inplace=True)
df.head()

Let's check the dimensionality of our dataframe.

In [ ]:
df.shape

# 2. Analyzing Individual Feature Patterns Using Visualization


### How to choose the right visualization method?
<p>When visualizing individual variables, it is important to first understand what type of variable you are dealing with. This will help us find the right visualization method for that variable.</p>


In [ ]:
# list the data types for each column
print(df.dtypes)

As you can see, all columns have correct types regarding their meanings. 

### Visualization

Let's downsample the series intervals.

In [ ]:
df_visual = df.copy()

df_visual = df_visual.loc[:, ["price BTC", "size BTC", "Saldo"]].resample("1H").agg({
    "price BTC": "first",
    "size BTC": "sum",
    "Saldo": "last"
})
df_visual.head()


<strong>matplotlib.pyplot</strong> is a state-based interface to matplotlib. It provides an implicit, MATLAB-like, way of plotting.<br>
It also opens figures on your screen, and acts as the figure GUI manager.

Now let's plot.

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 8))

axs[0].plot(df.index, df["price BTC"])
axs[0].set_title("price BTC")
axs[0].set_ylabel("Price")

axs[1].plot(df.index, df["size BTC"])
axs[1].set_title("size BTC")
axs[1].set_ylabel("Size")

axs[2].plot(df.index, df["Saldo"])
axs[2].set_title("Saldo")
axs[2].set_ylabel("Saldo")

plt.tight_layout()

plt.show()


From these charts we can follow the BTC asset "prices", "size" and "Saldo" getting a good overview of the price behavior.

Let's also look at the dependence of ASK and BID volume on Saldo

In [ ]:
sns.violinplot(data=df[['Saldo', 'size ASK']])

plt.xlabel('Saldo')
plt.ylabel('size ASK')
plt.title('Violinplot of Saldo and size ASK')

plt.show()

sns.violinplot(data=df[['Saldo', 'size BID']])

plt.xlabel('Saldo')
plt.ylabel('size BID')
plt.title('Violinplot of Saldo and size BID')

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

sns.violinplot(data=df[['Saldo', 'size ASK']], ax=ax[0])
ax[0].set_xlabel('Saldo')
ax[0].set_ylabel('size ASK')
ax[0].set_title('Violinplot of Saldo and size ASK')

# sns.violinplot(x='Saldo', y='size BID', data=df, ax=ax[1])
# ax[1].set_xlabel('Saldo')
# ax[1].set_ylabel('size BID')
# ax[1].set_title('Violinplot of Saldo and size BID')

sns.violinplot(data=df[['Saldo', 'size BID']], ax=ax[1])
ax[1].set_xlabel('Saldo')
ax[1].set_ylabel('size BID')
ax[1].set_title('Violinplot of Saldo and size BID')


plt.show()


## Correlation 

### What is a correlation?

A statistical measure known as <strong>correlation</strong> expresses how closely two variables are related linearly (meaning they change together at a constant rate). It's a typical technique for describing straightforward connections without explicitly stating cause and effect.

We can calculate the correlation between different variables. For instance, for types <code>int64</code> or <code>float64</code> using the method <code>corr()</code>:


Let's calculate the correlation for BTCBUSD currency paramters and its indicators.

In [ ]:
# find correlation
corr = df.corr()
corr

### How to interpret correlation results?

The magnitude of the correlation coefficient indicates the strength of the association. A correlation of -1.0 indicates a perfect negative correlation, and a correlation of 1.0 indicates a perfect positive correlation. If the correlation coefficient is greater than zero, it is a positive relationship. Conversely, if the value is less than zero, it is a negative relationship.

To better understand the concept of correlation we should visualize the strength of relationships between numerical variables using <b>correlation heatmaps</b>. 

### What is Correlation Heatmap? 

<strong>Correlation heatmaps</strong> are a type of plot that visualize the strength of relationships between numerical variables. 

Correlation plots are used to understand which variables are related to each other and the strength of this relationship.


In [ ]:
sns.heatmap(corr)

### How do you interpret a correlation heatmap?

Correlation ranges from -1 to +1. Values closer to 0 means there is no linear trend between the two variables. The closer the correlation is to 1, the more positively associated they are. The diagonal elements are always one. 

 From the heatmap, it can be seen that the most correlated among the indicators for the price of our cryptocurrency are the Volume and NATR parameters. However, their correlation values are not that high.

We will study correlation more precisely (Pearson correlation in-depth) at the end of the notebook.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1: </h1>

<p>Find the correlation between the BTC currency price and its size columns. Build heatmap to understand the resuluts more precisely.</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
# find correlation
corr = df[['price BTC', 'size BTC']].corr()
corr
```

</details>

In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
# build heatmap
sns.heatmap(corr, annot=True)
```

</details>


The Volume and NATR parameters are the two price indicators for our coin that have the highest correlation, as can be seen from the heatmap. Their correlation values are not very great, though.
It is also noticeable how the parameters NATR and TRANGE are highly correlated with Volume. They can be useful for determining his future performance.

## Continuous Numerical Variables

### What are Continuous numerical variables?

<p><strong>Continuous numerical variables</strong> are variables that may contain any value within some range. They can be of type <code>int64</code> or <code>float64</code>. A great way to visualize these variables is by using scatterplots with fitted lines.</p>

<p>In order to start understanding the <em>linear relationship</em> between an individual variable and the price, we can use <code>regplot</code> which plots the <strong>scatterplot</strong> plus the fitted regression line for the data.</p>

<pre>
<strong>A scatter plot</strong> is a type of plot or mathematical diagram using Cartesian coordinates to display values for typically two variables for a set of data.
</pre>

Let's see several examples of different linear relationships:


### Weak Linear Relationship


Firstly, we declare custom <code>regplot()</code> function responsible of plotting linear regression.

In [ ]:
def regplot(pd: pd.DataFrame, x: str, y: str) -> None:
    """ Return data plot and a linear regression model fit.
    """
    sns.regplot(x=x, y=y, data=pd)
    plt.ylim(0,)

    plt.xlabel(f"{x}, BTC")
    plt.ylabel(f"{y}, BTC")
    plt.show()

Let's find the scatterplot of "price BTC" and "size BTC". 

In [ ]:
# Open ETHBUSD price as potential predictor variable of BTCBUSD price
regplot(df, 'Saldo', 'size BTC')

<p>Volume does not seem like a good predictor of the price at all since the regression line is close to horizontal. Also, the data points are very scattered and far from the fitted line, showing lots of variability. Therefore, it's not a reliable variable.</p>

We can examine the correlation between BTC price and size and see that it's approximately 0.250284.


In [ ]:
df[['Saldo', 'size BTC']].corr()

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #2 a): </h1>

<p>Find the correlation "price BTC" and "Saldo".</p>
<p>Hint: if you would like to select those columns, use the following syntax: <code>df[['price BTC','Saldo']]</code>.  </p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
# find the correlation
df[["price BTC", "Saldo"]].corr()
```

</details>


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question #3 b):</h1>

<p>Given the correlation results between "price BTC" and "Saldo", do you expect a linear relationship?</p> 
<p>Verify your results using the function <code>regplot()</code>.</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 

<details><summary>Click here for the solution</summary>

```python

#There is a weak correlation between the variables. As such regression will not work well. We can see this using "regplot" to demonstrate this.

#Code: 
regplot(df, 'price BTC', 'Saldo')

```

</details>


### Categorical Variables

<p><strong>Categorical variables</strong> are variables that describe a 'characteristic' of a data unit, and are selected from a small group of categories. The categorical variables can have the type <code>object</code> or <code>int64</code>. A good way to visualize categorical variables is by using <strong>boxplots.</strong></p>


Let's create the following categorical values: 

In [ ]:
group_names = ["Low", "Medium", "High"]

Similiar to previous module, declare a function to create categorical variables for a given dataframe column.  

In [ ]:
def to_categorical(column: pd.Series, categories: List[str]) -> pd.Series:
    """ Return categorical variables for a given dataframe column.
    """
    bins = np.linspace(min(column), max(column), len(categories) + 1)
    return pd.cut(column, bins, labels=categories, include_lowest=True)

Let's use declared function on Saldo.

In [ ]:
df['Saldo_binned'] = to_categorical(df['Saldo'], group_names)
df[['Saldo', 'Saldo_binned']].head()

In [ ]:
df['Saldo_binned'].value_counts()

Great! Now we are one step closer to fully understand our dataset.

### Visualization of categorical variables

#### What is  a boxplot?

<strong><em>A boxplot</em></strong> is a graph that gives you a good indication of how the values in the data are spread out. We use a boxplot below to analyze the relationship between a categorical feature and a continuous feature.

Let's look at the relationship between <strong>"Saldo"</strong> and <strong>"Saldo_binned"</strong>. 

In [ ]:
sns.boxplot(x='Saldo', y='Saldo_binned', data=df)

Here we see that the distribution of price between these three categories are distinct enough to take in which catagery price will be.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Question 3:</h1>

<p>Create the same categories fo  Saldo and indicators size BTC, price BTC.</p>
<p>Hint: use declared <code>to_categorical()</code> function.</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
indicators = ['Saldo', 'price BTC','size BTC']
for indicator in indicators:
    df[f'{indicator}_binned'] = to_categorical(df[indicator], group_names)

# filter display categorical columns
df.filter(regex='_binned').head()

<details><summary>Click here for the solution</summary>

```python 
# create categories for currencies price
indicators = ['Saldo', 'price BTC','size BTC']
for indicator in indicators:
    df[f'{indicator}_binned'] = to_categorical(df[indicator], group_names)

# filter display categorical columns
df.filter(regex='_binned').head()

```

</details>


# 3. Descriptive Statistical Analysis


<p>Let's first take a look at the variables by utilizing a description method.</p>

<p>The <b>describe</b> function automatically computes basic statistics for all continuous variables. Any NaN values are automatically skipped in these statistics.</p>

This will show:

<ul>
    <li>the count of that variable</li>
    <li>the mean</li>
    <li>the standard deviation (std)</li> 
    <li>the minimum value</li>
    <li>the IQR (Interquartile Range: 25%, 50% and 75%)</li>
    <li>the maximum value</li>
<ul>


We can apply the method "describe" as follows:


In [ ]:
df.describe()

The default setting of "describe" skips variables of type object. We can apply the method "describe" on the variables of type 'object' as follows:


In [ ]:
df.describe(include=['category'])

### Value Counts


<p><strong>Value counts</strong> is a good way of understanding how many units of each characteristic/variable we have. We can apply the <code>value_counts()</code> method on the column <strong>"Saldo_binned"</strong>. Don’t forget the method <code>value_counts()</code> only works on pandas series, not pandas dataframes. As a result, we only include one bracket <code>df['Saldo_binned']</code>, not two brackets <code>df[['Saldo_binned']]</code>.</p>


In [ ]:
df['Saldo_binned'].value_counts()

We can convert the series to a dataframe as follows:


In [ ]:
df['Saldo_binned'].value_counts().to_frame()

Let's repeat the above steps but save the results to the dataframe <strong>"Saldo_binned_counts"</strong> and rename the column <strong>"Saldo_binned"</strong> to <strong>"value_counts"</strong>.

In [ ]:
Saldo_binned_counts = df['Saldo_binned'].value_counts().to_frame()
Saldo_binned_counts.rename(columns={'Saldo_binned': 'value_counts'}, inplace=True)
Saldo_binned_counts

Now let's rename the index to <strong>"Saldo_binned_counts"</strong>:

In [ ]:
Saldo_binned_counts.index.name = 'category'
Saldo_binned_counts

Now let's add indicators into this dataframe:

In [ ]:
category_count = Saldo_binned_counts.copy()
category_count.rename(columns = {'value_counts':'Saldo_value_counts'}, inplace = True)

for indicator in indicators:
    column_name = f'{indicator}_binned'
    # count number of values
    counts = df[column_name].value_counts().to_frame()
    # add new column to category_count
    category_count[f'{indicator}_value_counts'] = counts

category_count

# 4. Basics of Grouping


<p>The <code>groupby</code> method groups data by different categories. The data is grouped based on one or several variables, and analysis is performed on the individual groups.</p>

<p>For example, let's group by the variable <strong>"Saldo"</strong>. We see that there are 5 different categories of price.</p>


In [ ]:
df['Saldo_binned'].unique()

<p>If we want to know, on average, which type of <strong>"Saldo"</strong> is most valuable, we can group <strong>"Saldo"</strong> and then average them.</p>

<p>We can select the columns <code>'Saldo_binned'</code>, <code>'Saldo'</code> then assign it to the variable <strong>"df_group_one".</strong></p>


In [ ]:
df_group_one = df[['Saldo_binned', 'Saldo']]

We can then calculate the price for each of the different categories of data.


In [ ]:
# grouping results
df_group_one = df_group_one.groupby(['Saldo_binned'], as_index=True).mean()
df_group_one

<p>You can also group by others variables. For example, let's group by both <code>'Saldo_binned'</code> and <code>'size BTC_binned'</code>. We can store the results in the variable <code>'grouped_test_1'</code>.</p>


In [ ]:
# grouping results
df_gptest = df[['Saldo_binned','size BTC_binned','price BTC']]
grouped_test1 = df_gptest.groupby(['Saldo_binned','size BTC_binned'],as_index=False).mean()
grouped_test1

Also we can use a crossed table to see how many values correspond to each other in the table.

In [ ]:
crossed_table = pd.crosstab(df['Saldo_binned'], df['size BTC_binned'])
crossed_table

### Visualization of relationships 

Visualization methods show relationships and connections between the data or show correlations between two or more variables.

Execute the following to adjust astropy.

In [ ]:
astropy_mpl_style['axes.grid'] = False
plt.style.use(astropy_mpl_style)

Let's use a heat map to visualize the relationship between <strong>Saldo_binned</strong> <em>vs.</em> <strong>size BTC_binned</strong>.


In [ ]:
#use the grouped results
plt.pcolor(crossed_table, cmap='RdBu')
plt.colorbar()
plt.show()

<p>The heatmap plots the target variable <em>(price)</em> proportional to colour with respect to the variables <strong>'Saldo_binned'</strong> and <strong>'size BTC_binned'</strong> on the vertical and horizontal axis, respectively. This allows us to visualize  <strong>'Saldo_binned'</strong> and <strong>'size BTC_binned'.</strong></p>

<p>The default labels convey no useful information to us. Let's change that:</p>


In [ ]:
fig, ax = plt.subplots()
im = ax.pcolor(crossed_table, cmap='RdBu')

#label names
row_lables = crossed_table.columns.categories
col_labels = crossed_table.index

#move ticks and labels to the center
ax.set_xticks(np.arange(crossed_table.shape[1]) + 0.5, minor=False)
ax.set_yticks(np.arange(crossed_table.shape[0]) + 0.5, minor=False)

#insert labels
ax.set_xticklabels(col_labels, minor=False)
ax.set_yticklabels(row_lables, minor=False)

plt.xlabel("Saldo")
plt.ylabel("size BTC")

#rotate label if too long
plt.xticks(rotation=90)

fig.colorbar(im, label='Count')
plt.show()

<p>Visualization is very important in data science, and Python visualization packages provide great freedom. We will go more in-depth in a separate Python visualizations course.</p>

<p>The main question we want to answer in this module is <strong><em>"What are the main characteristics that have the most impact on the price of cryptocurrency?"</em></strong>.</p>

<p>To get a better measure of the important characteristics, we look at the correlation of our currency with indicators and other cryptocurrencies. In other words: <strong><em>how is the price dependent on other variables?</em></strong></p>


# 5. Correlation and Causation


<h4>What is correlation and causation?<h4>

<strong>Correlation</strong>: a measure of the extent of interdependence between variables.

<strong>Causation</strong>: the relationship between cause and effect between two variables.

<em><strong>It is important to know the difference between these two</strong></em>.

<li>Correlation does not imply causation.</li>
<li>Determining correlation is much simpler than the determining causation as causation may require independent experimentation.</li>

<h4>Pearson Correlation</h4>
<p><strong><em>The Pearson Correlation</em></strong> measures the linear dependence between two variables X and Y. In addition, It estimates the relationship strength between the two continuous variables.</p>
<p>The resulting coefficient is a value between <strong>-1</strong> and <strong>1</strong> inclusive, where:</p>
<ul>
    <li><strong>1</strong>: Perfect positive linear correlation.</li>
    <li><strong>0</strong>: No linear correlation, the two variables most likely do not affect each other.</li>
    <li><strong>-1</strong>: Perfect negative linear correlation.</li>
</ul>

The formula for Perason Correaltion between X and Y is represented as:

$$
r = \frac{n \sum \limits _{i=1} ^{n} xy - \sum \limits _{i=1} ^{n} x \sum \limits _{i=1} ^{n} y}{\sqrt{[n \sum \limits _{i=1} ^{n} x^2 - (\sum \limits _{i=1} ^{n} x)^2][n \sum \limits _{i=1} ^{n} y^2 - (\sum \limits _{i=1} ^{n} y)^2]}}
$$

<p>Pearson Correlation is the default method of the function <code>corr()</code>. Like before, we can calculate the Pearson Correlation of the <code>int64</code> or <code>'float64'</code> variables.</p>


In [ ]:
df.corr()

Sometimes we would like to know the significant of the correlation estimate.


#### <strong>P-value</strong>

#### <strong>What is this P-value?</strong> 

<em><strong>The P-value</strong></em> is the probability value that the correlation between these two variables is statistically significant. Normally, we choose a significance level of 0.05, which means that we are 95% confident that the correlation between the variables is significant.

By convention, when

<ul>
    <li><em>the p-value</em> is $<$ <strong>0.001</strong>: we say there is strong evidence that the correlation is significant.</li>
    <li><em>the p-value</em> is $<$ <strong>0.05</strong>: there is moderate evidence that the correlation is significant.</li>
    <li><em>the p-value</em> is $<$ <strong>0.1</strong>: there is weak evidence that the correlation is significant.</li>
    <li><em>the p-value</em> is $>$ <strong>0.1</strong>: there is no evidence that the correlation is significant.</li>
</ul>


Let's calculate the  Pearson Correlation Coefficient and P-value of our currency price and indicators.


In [ ]:
df_stats = pd.DataFrame({"indicator":[], "pearson": [], "p-value": []})

for indicator in indicators:
    pearson_coef, p_value = stats.pearsonr(df['price BTC'], df[indicator])
    df_stats.loc[len(df_stats.index)] = [indicator, pearson_coef, p_value]

df_stats.set_index('indicator', inplace=True)
df_stats

#### Conclusion:
<p>Since the <strong><em>p-value is $<$ 0.001</em></strong>, the correlation between our currency price and the others parameters is statistically significant, although the linear relationship isn't strong ar the correlation values are too small and approach zero.</p>


# 7. Durbin Watson Test

#### What Is the Durbin Watson Statistic? 

<strong>The Durbin Watson (DW) statistic</strong> is a test for autocorrelation in the residuals from a statistical model or regression analysis. The Durbin-Watson statistic will always have <em>a value ranging between 0 and 4</em>.

<li><strong>a value of 2.0</strong> indicates there is no autocorrelation detected in the sample.</li> 
<li><strong>values from 0 to less than 2</strong> point to positive autocorrelation</li>
<li><strong>values from 2 to 4</strong> means negative autocorrelation</li>

<br>
<p>
A rule of thumb is that DW test statistic values <em><strong> in the range of 1.5 to 2.5 are relatively normal</strong></em>. Values outside this range could, however, be a cause for concern. 
</p>

This test uses the following hypotheses:
<li><strong>$H_0$ (null hypothesis):</strong> There is no correlation among the residuals.</li>
<li><strong>$H_A$ (alternative hypothesis):</strong> The residuals are autocorrelated.</li>

Let's implement a function that creates regression models:

In [ ]:
def get_reg(x: pd.Series, y: pd.Series):
    """ Return regression model.
    """
    # to get intercept
    X = sm.add_constant(x)
    # fit the regression model
    reg = sm.OLS(y, X).fit()
    return reg

Now let's test this function and perform Durbin Watson Test.

In [ ]:
# independent
X = df['Saldo']
# dependent
y = df['price BTC']
reg = get_reg(X, y)

In [ ]:
print('DW test stats:', durbin_watson(resids=np.array(reg.resid)))

Now that we now how to calculate Durbin-Watson we can evaluate it for the main and indicators with other currencies.

In [ ]:
df_durbin = pd.DataFrame({'indicator':[], 'durbin-watson': []})

for indicator in indicators:
    # independent
    X = df["Saldo"]
    # dependent
    y = df['price BTC']
    reg = get_reg(X, y)
    dw = durbin_watson(resids=np.array(reg.resid))
    df_durbin.loc[len(df_durbin.index)] = [indicator, dw]

df_durbin.set_index('indicator', inplace=True)
df_durbin

According to the results of other indicators, we observe the same situation. The influence of indicators on the trend of changes in the price of cryptocurrency is not positively revealed.

Let's calculate Durbin-Watson for all available indicator pairs.

In [ ]:
variables = ["price BTC","Saldo"]

cols = [f"{variable}_dep" for variable in variables]
idxs = [f"{variable}_ind" for variable in variables]

dw_df = pd.DataFrame(columns=cols, index=idxs)

for (curr1, curr2) in itertools.permutations(variables, 2):
    
    X = df[f"{curr1}"]  # independent
    y = df[f"{curr2}"] # dependent
    # to get intercept
    X = sm.add_constant(X)
    # fit the regression model
    reg = sm.OLS(y, X).fit()
    dw = durbin_watson(resids=np.array(reg.resid))
    dw_df.loc[f"{curr1}_ind", f"{curr2}_dep"] = dw
    
dw_df

Replace NaN values with empty values.

In [ ]:
np.fill_diagonal(dw_df.values, ' ')
dw_df

#### Conclusion

Any value we choose will have a row for the independent value and a column for the dependent value.

if we look at the column of the dependent variable price BTC, we will see that no indicator of independent variables (indicators) is in the range of 1.5-2.5. This tells that the listed indicators cannot be a good prediction for the Saldo.

# 8. Granger-Causality Test

#### What is Granger-Causality test?

<strong>The Granger causality test</strong> is a statistical hypothesis test for determining whether one time series is useful in forecasting another. If the probability value is less than in our case <em>P-value level</em>, then the hypothesis would be rejected at that level.

#### What is the difference between correlation and Granger causality?

<strong><em>Correlation</em></strong> is a measure of linear dependence between two random variables. So no additional variables are involved in the calculation of the correlation between X and Z, and also, in principle these variables may be just random variables and not time series.

<strong><em>Granger causality</em></strong> is a concept of marginal predictability. So here the time dimension of the potential relationship between X and Z is important.

#### What is the null hypothesis in Granger Causality test?

<strong>The null hypothesis ($H_0$)</strong> for the test is that lagged x-values do not explain the variation in y. In other words, it assumes that $x_t$ doesn't Granger-cause $y_t$. Theoretically, you can run the Granger Test to find out if two variables are related at an instantaneous moment in time.

Let's visualize price trends movements through specific time period.

In [ ]:
x = df.head().index
y1 = df['price BTC'].head()
y2 = df['Saldo'].head()

# # Plot Line1 (Left Y Axis)
fig, ax1 = plt.subplots(1,1,figsize=(16,9), dpi= 80)
ax1.plot(x, y1, color='tab:red')

# # Plot Line2 (Right Y Axis)
# instantiate a second axes that shares the same x-axis
ax2 = ax1.twinx()
ax2.plot(x, y2, color='tab:blue')

# Decorations
# ax1 (left Y axis)
ax1.set_xlabel('Time', fontsize=20)
ax1.tick_params(axis='x', rotation=0, labelsize=12)
ax1.set_ylabel('price BTC', color='tab:red', fontsize=20)
ax1.tick_params(axis='y', rotation=0, labelcolor='tab:red' )
ax1.grid(alpha=.4)

# # ax2 (right Y axis)
ax2.set_ylabel('Saldo', color='tab:blue', fontsize=20)
ax2.tick_params(axis='y', rotation=0, labelsize=12, labelcolor='tab:blue')
ax2.set_title("Visualizing Leading Indicator Phenomenon", fontsize=22)

fig.tight_layout()
plt.show()

Let's import function implementing Granger Causality test from statsmodels module and declare Granger Causality test calculating function.

In [ ]:
def grangers_causation_matrix(data, maxlag, variables, test='ssr_chi2test', verbose=False):
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

Let's perform the Granger Causality test on the price of BTC and Saldo indicator.

In [ ]:
pd.options.display.float_format = '{:.4f}'.format
grangers_causation_matrix(df[['price BTC', 'Saldo']], 1, variables=['price BTC', 'Saldo'])

#### How are the P-values to be read?

If the P-value is less than 0.05, then, assuming a significance level of 0.05, we reject the null hypothesis that X does not generally cause Y.
Hence, the p-value for <strong>Saldo BTC_x</strong> and <strong>price BTC_y</strong> in the above table is equal to 0.0 determining that Saldo causes price BTC and that leads us to reject the null hypothesis.

<em>Therefore, it is likely that the Saldo movement will be useful in projecting price BTC.</em>

<p>
However, <strong>Saldo_x</strong> and <strong>price BTC_y</strong> have P-values of 0.0018. We cannot rule out the null hypothesis because the P-value does not equal or fall below 0.05.

<em>In other words, Saldo BTC can not be predicted from price BTC.</em>
</p>

## Let's calculate Granger Causality Test for many available indicators pairs.

In [ ]:
grangers_causation_matrix(df[['size BTC', 'price BTC']], 1, variables=['size BTC', 'price BTC'])

In [ ]:
grangers_causation_matrix(df[['size BTC', 'Saldo']], 1, variables=['size BTC', 'Saldo'])

### Thank you for completing this lab!

## Author

<a href="https://author.skills.network/instructors/danyil_zhupnyk">Danyil Zhupnyk</a><br>
<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2">Prof. Yaroslav Vyklyuk, DrSc, PhD</a><br>
<a href="https://author.skills.network/instructors/mariya_fleychuk">Prof. Mariya Fleychuk, DrSc, PhD</a><br>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2023-03-11        | 1.0     | D.Zhupnyk  | Created Lab                        |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>
